In [5]:
import svhnFileReader as sv
import numpy as np
import TrainSVHN as ts
import importlib

## loading files

In [89]:
# Dataset download and evaluation
# ===============================

# dependencies
importlib.reload(ts)
importlib.reload(sv)

files = ["train", "test", "extra"]

# download svhn files
for file in files:
    sv.maybeDownload(file)

# choose some random indieces
data_samples = np.random.randint(1, 400, size=20)
print(data_samples) # are they random?

# get file names and labels associated with random indieces
train_files, train_labels = sv.getLabels(
    'train/digitStruct.mat', data_samples)

# check the file names and labels
print(train_files)
print(train_labels)

# get the data in file names
data = sv.getImage(train_files, 'train/', shape=(100, 50))
print(data.shape) # what is the size? does it match?
print(sv.parseLabel(train_labels[0]))

# show some of the data
sv.showMultipleArraysHorizontally(data[:15], train_labels[:15], 3)

train already exists
test already exists
extra already exists
[159  84 266 168  10 101 265 156 224  85 173  70  44 275 160 207 232 305
 134 105]
['160.png', '85.png', '267.png', '169.png', '11.png', '102.png', '266.png', '157.png', '225.png', '86.png', '174.png', '71.png', '45.png', '276.png', '161.png', '208.png', '233.png', '306.png', '135.png', '106.png']
[[2.0, 10.0, 8.0], [1.0, 7.0], [2.0, 1.0], [4.0, 2.0], [2.0, 3.0], [3.0, 5.0], [5.0, 10.0], [5.0, 7.0], [2.0], [3.0, 5.0], [5.0, 9.0], [2.0, 3.0, 8.0, 6.0], [1.0, 10.0], [1.0, 7.0, 8.0], [8.0, 5.0], [9.0], [1.0, 3.0, 5.0, 8.0], [8.0, 7.0], [6.0, 8.0], [1.0, 4.0]]
(20, 50, 100, 3)
[0.0, 2.0, 10.0]


In [68]:
# Preprocessing
# =============

# dependencies
importlib.reload(ts)
importlib.reload(sv)

# configurations
dataset = 'train'
big_batch_size = 200
image_shape = (100,50)
pickle_file = dataset+"_preprocessed"

# read lots of files
struct_file = dataset+"/digitStruct.mat"
number_of_files = sv.getNumberOfFiles(struct_file)
data_samples = np.random.randint(0, number_of_files, number_of_files)
file_handle = open(pickle_file,"wb")

# iterate over data in big batches
for batch_start in range(0,number_of_files, big_batch_size):
    
    # read the .mat file and parse attributes of data files
    batch_indexes = data_samples[batch_start:batch_start+big_batch_size]
    print("Compeleted %d out of %d"%(batch_start+big_batch_size,number_of_files))

    file_names,train_labels = sv.getLabels(struct_file,batch_indexes)
    train_values = sv.getImage(file_names, dataset,shape=image_shape)

    # form and normalize
    pixel_depth = 255
    train_values = sv.scaleData(train_values,pixel_depth)
#     train_labels = sv.formLabels(train_labels)
    
    
    # save in file
    np.save(file_handle, train_values)
    np.save(file_handle, train_labels)
    
# always close the file
file_handle.close()


# Define generator
# ================
def generator(batch_size,file_name):
    file_handle = open(file_name, "rb")
    while True:

        # get data array
        try:
            data = np.load(file_handle)
        # if reached end of file
        except OSError:
            print("Pointer is at",file_handle.tell())
            # go to the beginning
            file_handle.seek(0)
            # and try loading again
            data = np.load(file_handle)

        # get label array
        labels = np.load(file_handle)
        
        # randomize
        number_of_datapoints = labels.shape[0]
        random_indexes = np.random.randint(1, number_of_datapoints, number_of_datapoints)
        data = data[random_indexes]
        labels = labels[random_indexes]
        del random_indexes
        
        # get batches        
        full_batches = number_of_datapoints//batch_size # few datapoints are going to waste here
        start_point = 0
        for batch_start in range(0,full_batches,batch_size):
            batch_data = data[batch_start:batch_start+batch_size]
            batch_labels = labels[batch_start:batch_start+batch_size]
            
            # yield both
            yield batch_data, batch_labels


Compeleted 200 out of 33402
Compeleted 400 out of 33402
Compeleted 600 out of 33402
Compeleted 800 out of 33402


KeyboardInterrupt: 

In [53]:

importlib.reload(ts)
network = ts.SVHNTrainer()
network.makeGraph()

def dataMaker(batch_size,dataset="train"):
    data_samples=np.random.randint(0,400,size=batch_size)
    train_files, train_labels = sv.getLabels(dataset+"/digitStruct.mat", data_samples)
    data = sv.getImage(train_files, 'train', shape=image_shape)
#     batch_image, batch_labels = mf.fixedSizeMultipleNumberRows(train_data,train_labels,digit_length+1,5)
    print(data.shape)
    batch_image = sv.scaleData(np.expand_dims(data,3),255)
    return batch_image,sv.toOnehot([len(label) for label in train_labels],5)

validation_data,validation_labels=dataMaker(9)
# test_data,test_labels=dataMaker(100)
# mf.showMultipleArraysHorizontally([test_data[i,0:28,0:140,0] for i in range(5)], test_labels,max_per_row=1)

# network.train(10000,dataMaker,100,100)

logit shape is [8, 56]
Seperated shapes are [[8, 6], [8, 10], [8, 10], [8, 10], [8, 10], [8, 10]]
loss shape is ()
(9, 50, 100, 3)
